# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_csv_file = "../WeatherPy/city_data_output.csv"

weather_df = pd.read_csv(output_csv_file)

weather_df = weather_df.drop(columns=["Unnamed: 0"])
weather_df



,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,bambous virieux,MU,79.05,69.0,40.0,14.97,-20.3428,57.7575
1,howard springs,AU,81.55,67.0,100.0,3.53,-12.4970,131.0470
2,port alfred,ZA,63.18,95.0,100.0,1.99,-33.5906,26.8910
3,iqaluit,CA,0.70,77.0,20.0,0.00,63.7506,-68.5145
4,busselton,AU,64.54,51.0,4.0,11.16,-33.6500,115.3333
...,...,...,...,...,...,...,...,...
626,xai-xai,MZ,78.28,68.0,3.0,6.78,-25.0519,33.6442
627,sabzevar,IR,83.48,20.0,40.0,16.11,36.2126,57.6819
628,vao,NC,74.89,75.0,79.0,20.31,-22.6667,167.4833
629,umm jarr,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
weather_df=weather_df.dropna()
weather_df


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,bambous virieux,MU,79.05,69.0,40.0,14.97,-20.3428,57.7575
1,howard springs,AU,81.55,67.0,100.0,3.53,-12.4970,131.0470
2,port alfred,ZA,63.18,95.0,100.0,1.99,-33.5906,26.8910
3,iqaluit,CA,0.70,77.0,20.0,0.00,63.7506,-68.5145
4,busselton,AU,64.54,51.0,4.0,11.16,-33.6500,115.3333
...,...,...,...,...,...,...,...,...
624,onega,RU,34.07,82.0,100.0,8.86,63.9061,38.1404
626,xai-xai,MZ,78.28,68.0,3.0,6.78,-25.0519,33.6442
627,sabzevar,IR,83.48,20.0,40.0,16.11,36.2126,57.6819
628,vao,NC,74.89,75.0,79.0,20.31,-22.6667,167.4833


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [5]:
print(min(humidity))

4.0


In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
target_city_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] < 80) & 
                                (weather_df["Cloudiness"] == 0) & weather_df["Windspeed"] <10, :]

temperature = (target_city_df["Temperature"] <= 80) & (target_city_df["Temperature"] > 70)
windspeed = target_city_df["Windspeed"] < 10
cloudiness = target_city_df["Cloudiness"] == 0

target_city_df=target_city_df[temperature & windspeed & cloudiness]

target_city_df



,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
161,port hedland,AU,78.85,65.0,0.0,8.05,-20.3167,118.5667
198,tura,IN,76.41,74.0,0.0,4.72,25.5198,90.2201
302,hamadan,IR,76.08,10.0,0.0,6.91,34.7992,48.5146


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = target_city_df[["City","Country","Latitude","Longitude"]].copy()

hotel_df


,City,Country,Latitude,Longitude
161,port hedland,AU,-20.3167,118.5667
198,tura,IN,25.5198,90.2201
302,hamadan,IR,34.7992,48.5146


In [21]:

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
161,port hedland,AU,-20.3167,118.5667,NaN
198,tura,IN,25.5198,90.2201,NaN
302,hamadan,IR,34.7992,48.5146,NaN


In [ ]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [25]:
# --- use a for loop to go through each city in the dataframe and make an API call ---

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"The nearest hotel in '{row['City']}' is {hotel_name}.")
        print("---------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'.")    
        print("-----------------------------------------------------")

The nearest hotel in 'port hedland' is Hedland Hotel.
---------------------------------------------------
The nearest hotel in 'tura' is Hotel Polo Orchid.
---------------------------------------------------
The nearest hotel in 'hamadan' is Parsian BuAli Hotel.
---------------------------------------------------


In [26]:
# --- remove null rows from dataframe ---
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# --- display final dataframe ---
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,port hedland,AU,-20.3167,118.5667,Hedland Hotel
1,tura,IN,25.5198,90.2201,Hotel Polo Orchid
2,hamadan,IR,34.7992,48.5146,Parsian BuAli Hotel


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [29]:
# --- add marker layer on top of heat map ---
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))